In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
df = pd.read_csv('train.csv')

In [25]:
df.shape

(404290, 6)

In [26]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [27]:
new_df = df.sample(30000)

In [28]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [29]:
new_df.duplicated().sum()

0

In [30]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
137611,Can objects in the Kuiper belt or the Oort clo...,Given the trillions of objects orbiting the su...
78475,Where will I get hydrogen gas?,What is hydrogen gas?
297550,How do road side blinkers/reflectors work? Wha...,What car handles the best at high speeds in th...
47701,Why is Spanish hard to learn?,Is it hard to learn Spanish? Why or why not?
195604,What should I do to prepare for my 12th boards...,How can I prepare for my 12th boards and nata ...


In [31]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [32]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [33]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
137611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195604,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355057,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245696,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
389312,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [35]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
137611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
78475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195604,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming X_train, y_train, X_test, y_test are already defined

# Initialize and train the model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Print precision, recall, and f1-score
report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])  # Update target names if necessary
print(report)

from scipy.stats import ttest_rel

# Calculate residuals
residuals = y_test - y_pred

# Perform a paired t-test
t_statistic, p_value = ttest_rel(y_test, y_pred)

# Print the results
print(f't-statistic: {t_statistic}')
print(f'p-value: {p_value}')

# Determine if null hypothesis is accepted or rejected
if p_value <= 0.05:
    print("Reject the null hypothesis: There is a significant difference between the predicted and actual values.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the predicted and actual values.")


Accuracy: 0.7455
              precision    recall  f1-score   support

     Class 0       0.74      0.92      0.82      3788
     Class 1       0.76      0.45      0.57      2212

    accuracy                           0.75      6000
   macro avg       0.75      0.68      0.69      6000
weighted avg       0.75      0.75      0.73      6000

t-statistic: 24.14992626767781
p-value: 4.76620689532695e-123
Reject the null hypothesis: There is a significant difference between the predicted and actual values.


In [38]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming X_train, y_train, X_test, y_test are already defined

# Initialize and train the model
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

# Make predictions
y_pred = xgb.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Print precision, recall, and f1-score
report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])  # Update target names if necessary
print(report)

from scipy.stats import ttest_rel

# Calculate residuals
residuals = y_test - y_pred

# Perform a paired t-test
t_statistic, p_value = ttest_rel(y_test, y_pred)

# Print the results
print(f't-statistic: {t_statistic}')
print(f'p-value: {p_value}')

# Determine if null hypothesis is accepted or rejected
if p_value <= 0.05:
    print("Reject the null hypothesis: There is a significant difference between the predicted and actual values.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the predicted and actual values.")


Accuracy: 0.7258333333333333
              precision    recall  f1-score   support

     Class 0       0.73      0.90      0.80      3788
     Class 1       0.71      0.44      0.54      2212

    accuracy                           0.73      6000
   macro avg       0.72      0.67      0.67      6000
weighted avg       0.72      0.73      0.71      6000

t-statistic: 21.85035848452182
p-value: 6.6486220250450374e-102
Reject the null hypothesis: There is a significant difference between the predicted and actual values.
